# PFAS Tender Analysis
## Analyzing institutional PFAS communication and procurement patterns

This notebook analyzes:
- Budget allocation patterns
- Geographic distribution
- Temporal trends
- Text mining for framing analysis
- Organization networks

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data

In [ ]:
# Load tender data
df = pd.read_csv('pfas_analysis_ready/tenders_with_text.csv')

# Convert dates
df['publication_date'] = pd.to_datetime(df['publication_date'])
df['deadline'] = pd.to_datetime(df['deadline'])

print(f"Loaded {len(df)} PFAS tenders")
df.head()

## 2. Descriptive Statistics

In [ ]:
# Basic stats
print("=" * 80)
print("PFAS TENDER STATISTICS")
print("=" * 80)
print(f"\nTotal tenders: {len(df)}")
print(f"Date range: {df['publication_date'].min()} to {df['publication_date'].max()}")
print(f"\nAwarded: {df['is_awarded'].sum()} ({df['is_awarded'].sum()/len(df)*100:.1f}%)")
print(f"Completed: {df['is_completed'].sum()} ({df['is_completed'].sum()/len(df)*100:.1f}%)")
print(f"European: {df['is_european'].sum()} ({df['is_european'].sum()/len(df)*100:.1f}%)")

# Budget stats
budget_df = df[df['estimated_budget'].notna()]
print(f"\nBudget information available: {len(budget_df)} tenders")
print(f"Total estimated value: €{budget_df['estimated_budget'].sum():,.0f}")
print(f"Average budget: €{budget_df['estimated_budget'].mean():,.0f}")
print(f"Median budget: €{budget_df['estimated_budget'].median():,.0f}")

## 3. Temporal Analysis

In [ ]:
# Tenders over time
df['year'] = df['publication_date'].dt.year
df['month'] = df['publication_date'].dt.to_period('M')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# By year
df['year'].value_counts().sort_index().plot(kind='bar', ax=ax1, color='steelblue')
ax1.set_title('PFAS Tenders by Year', fontsize=14, fontweight='bold')
ax1.set_xlabel('Year')
ax1.set_ylabel('Number of Tenders')

# By month (last 24 months)
monthly = df['month'].value_counts().sort_index().tail(24)
monthly.plot(kind='line', ax=ax2, marker='o', color='steelblue')
ax2.set_title('PFAS Tenders by Month (Last 24 Months)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Month')
ax2.set_ylabel('Number of Tenders')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Geographic Distribution

In [ ]:
# Top locations
top_locations = df['location'].value_counts().head(10)

plt.figure(figsize=(12, 6))
top_locations.plot(kind='barh', color='coral')
plt.title('Top 10 Locations for PFAS Tenders', fontsize=14, fontweight='bold')
plt.xlabel('Number of Tenders')
plt.ylabel('Location')
plt.tight_layout()
plt.show()

print("\nTop 10 Locations:")
for loc, count in top_locations.items():
    print(f"  {count:2d}x {loc}")

## 5. Organizational Analysis

In [ ]:
# Top organizations
top_orgs = df['organization'].value_counts().head(15)

plt.figure(figsize=(12, 8))
top_orgs.plot(kind='barh', color='mediumseagreen')
plt.title('Top 15 Organizations Procuring PFAS Work', fontsize=14, fontweight='bold')
plt.xlabel('Number of Tenders')
plt.ylabel('Organization')
plt.tight_layout()
plt.show()

# Organization types
def categorize_org(org):
    if pd.isna(org):
        return 'Unknown'
    org_lower = org.lower()
    if 'gemeente' in org_lower:
        return 'Municipality'
    elif 'provincie' in org_lower:
        return 'Province'
    elif 'waterschap' in org_lower or 'hoogheemraadschap' in org_lower:
        return 'Water Board'
    elif 'rijk' in org_lower or 'ministerie' in org_lower:
        return 'National Government'
    elif 'brandweer' in org_lower:
        return 'Fire Department'
    else:
        return 'Other'

df['org_type'] = df['organization'].apply(categorize_org)

plt.figure(figsize=(10, 6))
df['org_type'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('PFAS Tenders by Organization Type', fontsize=14, fontweight='bold')
plt.ylabel('')
plt.show()

## 6. Budget Analysis

In [ ]:
budget_df = df[df['estimated_budget'].notna()].copy()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Budget distribution
budget_df['estimated_budget'].hist(bins=20, ax=ax1, color='gold', edgecolor='black')
ax1.set_title('Budget Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Budget (€)')
ax1.set_ylabel('Frequency')

# Budget by organization type
budget_by_org = budget_df.groupby('org_type')['estimated_budget'].sum().sort_values(ascending=False)
budget_by_org.plot(kind='bar', ax=ax2, color='gold', edgecolor='black')
ax2.set_title('Total Budget by Organization Type', fontsize=14, fontweight='bold')
ax2.set_xlabel('Organization Type')
ax2.set_ylabel('Total Budget (€)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nBudget by Organization Type:")
for org_type, total in budget_by_org.items():
    print(f"  {org_type}: €{total:,.0f}")

## 7. Contract Type Analysis

In [ ]:
# Contract types
contract_types = df['contract_type'].value_counts()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Count
contract_types.plot(kind='bar', ax=ax1, color='skyblue', edgecolor='black')
ax1.set_title('PFAS Tenders by Contract Type', fontsize=14, fontweight='bold')
ax1.set_xlabel('Contract Type')
ax1.set_ylabel('Number of Tenders')
ax1.tick_params(axis='x', rotation=45)

# Procedure types
procedure_types = df['procedure_type'].value_counts().head(5)
procedure_types.plot(kind='bar', ax=ax2, color='lightcoral', edgecolor='black')
ax2.set_title('Top 5 Procedure Types', fontsize=14, fontweight='bold')
ax2.set_xlabel('Procedure Type')
ax2.set_ylabel('Number of Tenders')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 8. Text Analysis - PFAS Mentions

In [ ]:
# PFAS mentions distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

df['pfas_mentions'].hist(bins=20, ax=ax1, color='salmon', edgecolor='black')
ax1.set_title('Distribution of PFAS Mentions per Tender', fontsize=14, fontweight='bold')
ax1.set_xlabel('Number of PFAS Mentions')
ax1.set_ylabel('Frequency')

# PFAS mentions over time
pfas_by_year = df.groupby('year')['pfas_mentions'].mean()
pfas_by_year.plot(kind='line', marker='o', ax=ax2, color='salmon', linewidth=2)
ax2.set_title('Average PFAS Mentions per Tender Over Time', fontsize=14, fontweight='bold')
ax2.set_xlabel('Year')
ax2.set_ylabel('Average PFAS Mentions')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAverage PFAS mentions per tender: {df['pfas_mentions'].mean():.2f}")
print(f"Median PFAS mentions: {df['pfas_mentions'].median():.0f}")
print(f"Max PFAS mentions: {df['pfas_mentions'].max():.0f}")

## 9. Keyword Analysis

In [ ]:
# Extract all keywords
from collections import Counter

all_keywords = []
for keywords in df['keywords'].dropna():
    all_keywords.extend([k.strip() for k in keywords.split(',') if k.strip()])

keyword_counts = Counter(all_keywords)
top_keywords = dict(keyword_counts.most_common(15))

plt.figure(figsize=(12, 6))
plt.barh(list(top_keywords.keys()), list(top_keywords.values()), color='mediumpurple')
plt.title('Top 15 Keywords in PFAS Tenders', fontsize=14, fontweight='bold')
plt.xlabel('Frequency')
plt.ylabel('Keyword')
plt.tight_layout()
plt.show()

print("\nTop 15 Keywords:")
for keyword, count in keyword_counts.most_common(15):
    print(f"  {count:2d}x {keyword}")

## 10. Export Summary for Paper

In [ ]:
# Create summary statistics for paper
summary_stats = {
    'total_tenders': len(df),
    'date_range': f"{df['publication_date'].min().date()} to {df['publication_date'].max().date()}",
    'awarded_pct': f"{df['is_awarded'].sum()/len(df)*100:.1f}%",
    'european_pct': f"{df['is_european'].sum()/len(df)*100:.1f}%",
    'total_budget': f"€{budget_df['estimated_budget'].sum():,.0f}",
    'avg_budget': f"€{budget_df['estimated_budget'].mean():,.0f}",
    'top_organization': df['organization'].value_counts().index[0],
    'top_location': df['location'].value_counts().index[0],
    'avg_pfas_mentions': f"{df['pfas_mentions'].mean():.2f}",
}

print("\n" + "="*80)
print("SUMMARY STATISTICS FOR PAPER")
print("="*80)
for key, value in summary_stats.items():
    print(f"{key}: {value}")

# Save to JSON
with open('pfas_analysis_ready/summary_stats.json', 'w') as f:
    json.dump(summary_stats, f, indent=2)

print("\n✅ Summary statistics saved to summary_stats.json")